## Train 3d nodule detector with LUNA16 dataset

In [1]:
INPUT_DIR='../../input/nodules/'
OUTPUT_DIR='../../output/lung-cancer/04/'
MODEL_FILE='../../output/lung-cancer/03-ok/tf-checkpoint-best9615'
IMAGE_DIMS = (50,50,50,1)

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import sklearn
import os
import glob

from modules.logging import logger
import modules.utils as utils
from modules.utils import Timer
import modules.logging
import modules.cnn as cnn
import modules.ctscan as ctscan

## Test

### Prepare CNN model

In [3]:
logger.info('Prepare CNN for training')
network = cnn.net_nodule3d_swethasubramanian(IMAGE_DIMS)
model = cnn.prepare_cnn_model(network, OUTPUT_DIR, model_file=MODEL_FILE)

2017-03-28 00:36:58,355 INFO Prepare CNN for training
2017-03-28 00:36:58,575 INFO Prepare CNN
2017-03-28 00:36:58,588 INFO Preparing output dir
2017-03-28 00:36:58,589 INFO Initializing network...


InternalError: Dst tensor is not initialized.
	 [[Node: Adam/zeros_13 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [1500000,64] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'Adam/zeros_13', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/io

### Evaluate results

In [ ]:
logger.info('Evaluate dataset')
cnn.evaluate_dataset(INPUT_DIR + 'nodules-test.h5', model, batch_size=12, confusion_matrix=True)